# Mise à disposition du jeu de données stroke_dataset via une API REST

## Description du dataset

Le jeu de données utilisé provient de Kaggle : [Stroke Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset).  
Il contient des données de patients avec différentes caractéristiques médicales et sociales, ainsi que l'information si le patient a subi un accident vasculaire cérébral (AVC) ou non.

Télécharger les données et ajouter les dans un dossier data/.

Les colonnes des données sont :  
- `id` : Identifiant unique du patient  
- `gender` : Sexe  
- `age` : Âge  
- `hypertension` : Présence d'hypertension (0 ou 1)  
- `heart_disease` : Présence de maladie cardiaque (0 ou 1)  
- `ever_married` : Statut marital  
- `work_type` : Type d'emploi  
- `Residence_type` : Urbaine ou rurale  
- `avg_glucose_level` : Moyenne du taux de glucose  
- `bmi` : Indice de masse corporelle  
- `smoking_status` : Statut tabagique  
- `stroke` : Présence d'AVC (0 ou 1)

## Projet

Vous devez exposer les données patients du jeu de données via une API REST afin que les données soit utilisables par d'autres équipes (médecins, data science, étude, etc.).

Cette API REST sera développée avec FastAPI et les spécifications sont les suivantes :
| Méthode | Endpoint                                      | Fonctionnalité                                                                                                    |
| ------- | --------------------------------------------- | ----------------------------------------------------------------------------------------------------------------- |
| `GET`   | `/patients/{id}`                              | Récupère les détails d’un patient donné (via son identifiant unique)                                              |
| `GET`   | `/patients?stroke=1&gender=Female&max_age=60` | Renvoie les patients filtrés selon plusieurs critères : AVC (oui/non), genre, âge maximal                         |
| `GET`   | `/stats/`                                     | Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes, etc.) |



---

## Quelques définitions


1. Qu’est-ce qu’une API REST ?

- API signifie Application Programming Interface (Interface de Programmation d’Application). C’est un ensemble de règles et de protocoles qui permettent à des logiciels de communiquer entre eux.
- REST signifie Representational State Transfer. C’est un style architectural pour concevoir des services web.
Il en existe d'autres mais REST est celui que vous rencontrerez le plus souvent.
- Vous avez utilisé une API REST via l'API Google Books.

- A quoi sert une API REST ?

    - Permet à différentes applications de communiquer facilement, même si elles sont écrites dans des langages différents.
    - Permet d’accéder à des services distants (ex : bases de données, services web) de manière standardisée.
    - Facilite la création d’applications modulaires et évolutives (front-end, back-end, mobile, etc.)

2. Principes clés d’une API REST

- a. Utilisation du protocole HTTP
Les échanges entre client et serveur utilisent des méthodes HTTP standard comme :

    - GET : pour récupérer des données
    - POST : pour envoyer ou créer des données
    - PUT : pour mettre à jour des données
    - DELETE : pour supprimer des données

- b. Accès aux ressources via des URLs

Chaque ressource (par exemple un livre, un utilisateur) est accessible via une URL unique.

Exemple fictif:
    https://api.example.com/books/123 pour accéder au livre d’identifiant 123.

- c. Stateless (sans état)

Le serveur ne conserve aucune information sur le client entre deux requêtes. Chaque requête doit contenir toutes les informations nécessaires.

- d. Représentations des données

Les données sont envoyées et reçues généralement en format JSON ou XML, qui sont faciles à lire et à manipuler.

- e. Utilisation de codes status HTTP

Chaque réponse du serveur est accompagnée d’un code HTTP indiquant le résultat de la requête. (cf [liste des codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status))

## Outils utilisés
1. FastAPI

FastAPI est un framework Python moderne, rapide et très utilisé dans le milieu professionnel pour construire des API REST. 

Il permet :
- une définition simple des routes et des paramètres  
- La génération automatique de documentation interactive (Swagger UI)  
- FastAPI lit les requêtes entrantes, les traite avec ton code Python, et retourne une réponse HTTP (en JSON).

2. Uvicorn : exécute l'application FastAPI

- Uvicorn est un serveur ASGI (Asynchronous Server Gateway Interface) : c'est une interface standard pour gérer les requêtes de manière asynchrone et performante, notamment utile pour les applications modernes.
- Il attend les requêtes HTTP (par exemple depuis un navigateur), les transmet à FastAPI, et renvoie la réponse.
- Uvicorn permet à l'API de fonctionner : sans Uvicorn ou un autre serveur, FastAPI ne peut pas fonctionner.


3. Swagger UI : l’interface de doc et test interactive

- Swagger UI est généré automatiquement par FastAPI.
- C’est une interface web qui permet de :
    - Voir toutes les routes disponibles dans l'API
    - Tester les routes en envoyant des requêtes sans écrire de code (bouton try it out)
    - Voir les paramètres attendus et les formats de réponse
    
4. Résumé des interactions

- Tester la route de base de l'API grâce à la commande :
```bash
    poetry run fastapi dev stroke_api/main.py
```

--> Qu'est-ce qu'il se passe derrière cette commande ?

- Uvicorn démarre un serveur local
- FastAPI génère automatiquement une interface : Swagger UI, accessible sur http://127.0.0.1:8000/docs qui affiche toutes les routes définies dans le code python FastAPI
- Quand on clique sur "Try it out" dans Swagger UI, Swagger envoie une requête HTTP au serveur (ici Uvicorn)
- Le serveur (Uvicorn) la reçoit, l’envoie à FastAPI, qui traite et renvoie une réponse
- Swagger UI affiche la réponse de l’API (par ex : liste de patients)

---

Import des bibliothèques utiles au projet

In [1]:
import pandas as pd
import fastapi as fp
import plotly_express as px

## 1. Prétraitement des données / Data preprocessing

Les données réelles sont rarement prêtes à être utilisées directement. Elles peuvent contenir des erreurs, des valeurs manquantes, des doublons, des formats incohérents, ou ne pas être adaptées au modèle ou au système cible.

Le prétraitement consiste à nettoyer, structurer et transformer les données brutes avant de les exploiter dans un projet (modèle IA, API, visualisation, etc.).

Vous avez déjà prétraité des données, petit rappel des éléments sur lesquels travailler dans un prétraitment classique et les méthodes pandas qu'il est possible d'utiliser pour les différentes étapes (des exemples d'utilisation des méthodes pandas sont disponibles dans la doc) : 
- explorer les données pour identifier les types de données, valeurs manquantes, incohérence ([info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html), [dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html))
- adapter les types si nécessaire ([astypes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html))
- identifier les doublons et les supprimer s'il y en a ([duplicated](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html), [drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html))
- traiter les valeurs manquantes s'il y en a ([fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html), [dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html), [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html))
- identifier les incohérences éventuelles (valeurs aberrantes/outliers) en vérifiant si les valeurs min, max, moyennes sont raisonnables (recherche internet si nécessaire) ([describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)), et les traiter.
- Traiter les valeurs aberrantes si vous en détectez ([loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) pour récupérer les lignes qui répondent à une certaine condition, cf exemple ci-dessous)


**Exemple df.loc :**

Récupérer toutes les lignes de df telles que la valeur de "nom de colonne" >= 0

```df_subset = df.loc[stroke_data_df['nom de colonne'] >= 0]```



---
### **TODO**
1.a. Prétraiter les données du dataset.


1.b. Documenter dans le README.md :
- Les étapes de prétraitement,
- Justification des choix concernant le traitement des valeurs manquantes (si besoin),
- Liste des valeurs raisonnables utilisées pour détecter les valeurs aberrantes, 
- Justification des choix pour traiter les valeurs aberrantes (si besoin).

2.a. Chercher des infos sur le format de fichier parquet et indiquer les sources consultées : 
- Différence principale avec le format csv ?
- Dans quels cas l'utiliser ? 
- Pourquoi c'est un format adapté aux gros volumes de données ?



2.b. Sauvegarder les données prétraiteées dans un fichier parquet ([to_parquet](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html)).


---
## 1. Prétraitement

 ### 1.a Prétraitement des données

In [2]:
# Prétraitement de données

df = pd.read_csv("../data/healthcare-dataset-stroke-data.csv")

df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB
None


On voit qu'il y a des données manquantes dans BMI. Les colonnes "ever_married","hypertension","heart disease","residence type" et "stroke" devrait être en boolean.
"work type" et "smoking status" en categorical ou string (plusieurs choix)

In [4]:
print(df.describe())

                 id          age  hypertension  heart_disease  \
count   5110.000000  5110.000000   5110.000000    5110.000000   
mean   36517.829354    43.226614      0.097456       0.054012   
std    21161.721625    22.612647      0.296607       0.226063   
min       67.000000     0.080000      0.000000       0.000000   
25%    17741.250000    25.000000      0.000000       0.000000   
50%    36932.000000    45.000000      0.000000       0.000000   
75%    54682.000000    61.000000      0.000000       0.000000   
max    72940.000000    82.000000      1.000000       1.000000   

       avg_glucose_level          bmi       stroke  
count        5110.000000  4909.000000  5110.000000  
mean          106.147677    28.893237     0.048728  
std            45.283560     7.854067     0.215320  
min            55.120000    10.300000     0.000000  
25%            77.245000    23.500000     0.000000  
50%            91.885000    28.100000     0.000000  
75%           114.090000    33.100000     0

Rien d'anormal dans ces données numériques.

In [5]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
5105    False
5106    False
5107    False
5108    False
5109    False
Length: 5110, dtype: bool

Aucun doublon.

On va regarder les valeurs uniques pour les colonnes d'object.

In [6]:
print(pd.unique(df['work_type']))
print(pd.unique(df['Residence_type']))
print(pd.unique(df['smoking_status']))
print(pd.unique(df['gender']))


['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
['Urban' 'Rural']
['formerly smoked' 'never smoked' 'smokes' 'Unknown']
['Male' 'Female' 'Other']


On va convertir les différents type de données comme écrit précédemment. Les colonnes "ever_married","hypertension","heart disease" et "stroke" devrait être en boolean.
"work type", "residence type", "smoking status" et "gender" en categorical ou string (plusieurs choix)
On voit aussi que les valeurs ne sont pas toutes en majuscules, certaines ont des espaces, d'autre non.
On décide de tout écrire en majuscule et de remplacer les _ par des espaces.

In [7]:
df["ever_married"] = df["ever_married"].map({"Yes": True, "No": False})
df["ever_married"] = df["ever_married"].astype("bool")
df["gender"] = df["gender"].astype("category")
df["hypertension"] = df["hypertension"].astype("bool")
df["heart_disease"] = df["heart_disease"].astype("bool")
df["stroke"] = df["stroke"].astype("bool")
df["work_type"] = df["work_type"].map({"Govt_job": "Government job", "children": "Children", "Never_worked":"Never worked", "Private":"Private", "Self-employed":"Self-employed"})
df["work_type"] = df["work_type"].astype("category")
df["Residence_type"] = df["Residence_type"].astype("category")
df["smoking_status"] = df["smoking_status"].map({"formerly smoked": "Formerly smoked", "never smoked": "Never smoked", "smokes":"Smokes", "Unknown":"Unknown"})
df["smoking_status"] = df["smoking_status"].astype("category")
df.dtypes

id                      int64
gender               category
age                   float64
hypertension             bool
heart_disease            bool
ever_married             bool
work_type            category
Residence_type       category
avg_glucose_level     float64
bmi                   float64
smoking_status       category
stroke                   bool
dtype: object

Hypothèse : Le taux de glucose dans le sang a un lien avec le BMI. Si c'est le cas, on peut utiliser le taux de glucose pour estimer les valeurs manquantes dans la colonne de BMI.

In [8]:
df['glucose_normalized'] = (df['avg_glucose_level'] - df['avg_glucose_level'].min()) / (df['avg_glucose_level'].max() - df['avg_glucose_level'].min())
df['bmi_normalized'] = (df['bmi'] - df['bmi'].min()) / (df['bmi'].max() - df['bmi'].min())
fig = px.histogram(df, x=["glucose_normalized", "bmi_normalized"], barmode="group")
fig.show()

On peut observer une certaine corrélation entre le taux de glucose et la BMI.
On calcule le coefficient de variation de la BMI et du glucose.

In [9]:
CV_glucose = df["avg_glucose_level"].std() / df["avg_glucose_level"].mean()*100
CV_bmi = df["bmi"].std() / df["bmi"].mean()*100
print(f"{round(CV_glucose)}%", f"{round(CV_bmi)}%")

43% 27%


En prenant en considération les coefficients de variation assez haut, on va donc utiliser la médiane des 2 valeurs plutôt que la moyenne.
En divisant la médiane de la BMI par la médiane du taux de glucose, on obtient un coefficient multiplicateur qui va nous permettre de créer une fonction afin d'estimer une valeur pour la BMI en fonction du taux de glucose.

In [10]:
gluc_median = df["avg_glucose_level"].median()
bmi_median = df["bmi"].median()
func = bmi_median / gluc_median
print(gluc_median, bmi_median, func)

91.88499999999999 28.1 0.3058170539261033


In [11]:
coef = df["bmi"].median() / df["avg_glucose_level"].median()

# Remplacer les NaN dans 'bmi'

df.loc[df["bmi"].isnull(), "bmi"] = (df.loc[df["bmi"].isnull(), "avg_glucose_level"] * coef)
df["bmi"] = df["bmi"].round(1)

# Drop les colonnes "normalized"

df = df.drop(columns=["bmi_normalized", "glucose_normalized"])

### 2. Informations sur le type de fichier "parquet"

#### 2.a Le format "parquet"

- Différence principale avec le format csv ?
Le csv est une format texte simple, sauvegarde les données par ligne, séparation virgule.
Le parquet est une format binaire colonnaire, qui stocke les données colonne par colonne, avec compression et conversation des types de données.
Le parquet est beaucoup plus rapide car il est capable de lire que certain colonnes à la fois sans lire les tout les données. Les fichiers sont beaucoup plus petits (grâce à la compression et au stockage colonnaire). Par contre, un CSV est plus simple à lire et partager, mais plus volumineux et plus lent sur les grand jeux de données.

- Dans quels cas l'utiliser ? 
Quant on travaille avec les données massives (machine learning, data engineering) ou on veut lire que certaines colonnes rapidement, et aussi pour archiver ou transfporter de gros ensembles des données en minimalisant la taille.

- Pourquoi c'est un format adapté aux gros volumes de données ?
Le stockage colonnaire permet de lire uniquement les colonnes nécessaires, donc on gagne le temps et des entrées/sorties disque sont reduit.
Le compression reduit la taille de fichiers (moins de stockage, transfert plus rapide)
Conservation des types : pas besoin de parser/convertir les valeurs (contrairement au CSV), ce qui accélère le traitement.

#### 2.b Enregistrement au format parquet

In [12]:
df.to_parquet('../data/healthcare-dataset-stroke-data-clean.parquet', index=False)

-----
## Développement de l'API

A présent que les données sont propres, on peut débuter la création de l'API.

Pour cela, vous allez avoir besoin de quelques fonctions permettant de filtrer les données.

Vous allez les définir ci-dessous, ce qui vous permettra de les tester puis les fonctions seront reportées dans le fichier filters.py.

## Route `/patients/`
- Cette route retourne une liste filtrée de patients
- On souhaite pouvoir filtrer par `gender`, `stroke` ou `max_age`

L'objectif est ici de définir une fonction python qui prend en entrée les paramètres optionnels : _gender_, *stroke*, *max_age* et qui renvoie un dictionnaire filtré des données.

On décompose la rédaction de cette fonction en plusieurs étapes. 

Dans un premier temps, écrire et tester les filtres que l'on souhaite appliquer sur les données (utiliser [loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html))

In [13]:
# Filtrer le dataframe pour ne garder que les patients pour lesquels "stroke=1"
df.loc[df["stroke"] == 1]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,False,True,True,Private,Urban,228.69,36.6,Formerly smoked,True
1,51676,Female,61.0,False,False,True,Self-employed,Rural,202.21,61.8,Never smoked,True
2,31112,Male,80.0,False,True,True,Private,Rural,105.92,32.5,Never smoked,True
3,60182,Female,49.0,False,False,True,Private,Urban,171.23,34.4,Smokes,True
4,1665,Female,79.0,True,False,True,Self-employed,Rural,174.12,24.0,Never smoked,True
...,...,...,...,...,...,...,...,...,...,...,...,...
244,17739,Male,57.0,False,False,True,Private,Rural,84.96,36.7,Unknown,True
245,49669,Female,14.0,False,False,False,Children,Rural,57.93,30.9,Unknown,True
246,27153,Female,75.0,False,False,True,Self-employed,Rural,78.80,29.3,Formerly smoked,True
247,34060,Male,71.0,True,False,True,Self-employed,Rural,87.80,26.9,Unknown,True


In [14]:
# Filtrer les données pour ne garder que les patients pour lesquels "gender="male"
df.loc[df["gender"] == "Male"]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,False,True,True,Private,Urban,228.69,36.6,Formerly smoked,True
2,31112,Male,80.0,False,True,True,Private,Rural,105.92,32.5,Never smoked,True
5,56669,Male,81.0,False,False,True,Private,Urban,186.21,29.0,Formerly smoked,True
6,53882,Male,74.0,True,True,True,Private,Rural,70.09,27.4,Never smoked,True
13,8213,Male,78.0,False,True,True,Private,Urban,219.84,67.2,Unknown,True
...,...,...,...,...,...,...,...,...,...,...,...,...
5097,64520,Male,68.0,False,False,True,Self-employed,Urban,91.68,40.8,Unknown,False
5098,579,Male,9.0,False,False,False,Children,Urban,71.88,17.5,Unknown,False
5099,7293,Male,40.0,False,False,True,Private,Rural,83.94,25.7,Smokes,False
5100,68398,Male,82.0,True,False,True,Self-employed,Rural,71.97,28.3,Never smoked,False


In [15]:
# Filtrer les données pour ne garder que les patients tels que "age <= max_age"
df.loc[df["age"] <= df['age'].max()]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,False,True,True,Private,Urban,228.69,36.6,Formerly smoked,True
1,51676,Female,61.0,False,False,True,Self-employed,Rural,202.21,61.8,Never smoked,True
2,31112,Male,80.0,False,True,True,Private,Rural,105.92,32.5,Never smoked,True
3,60182,Female,49.0,False,False,True,Private,Urban,171.23,34.4,Smokes,True
4,1665,Female,79.0,True,False,True,Self-employed,Rural,174.12,24.0,Never smoked,True
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,True,False,True,Private,Urban,83.75,25.6,Never smoked,False
5106,44873,Female,81.0,False,False,True,Self-employed,Urban,125.20,40.0,Never smoked,False
5107,19723,Female,35.0,False,False,True,Self-employed,Rural,82.99,30.6,Never smoked,False
5108,37544,Male,51.0,False,False,True,Private,Rural,166.29,25.6,Formerly smoked,False


Appliquer successivement les 3 filtres au sein d'une fonction qui prend en entrée le dataframe, _stroke_, _gender_, _max_age_ et qui renvoie une liste de dictionnaire de patients (utiliser la méthode pandas [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html)).

Exemple
```
[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1},
 {'id': 31112,
  'gender': 'Male',
  'age': 80.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1}]
  ```

In [16]:
def filter_patient(df_value, max_age, gender, stroke):
    df_filtered = df_value[(df_value["age"] <= max_age) & (df_value["gender"] == gender) & (df_value["stroke"] == stroke)]

    return df_filtered.to_dict('records')

filter_patient(df, 80, "Male", 0)

[{'id': 30669,
  'gender': 'Male',
  'age': 3.0,
  'hypertension': False,
  'heart_disease': False,
  'ever_married': False,
  'work_type': 'Children',
  'Residence_type': 'Rural',
  'avg_glucose_level': 95.12,
  'bmi': 18.0,
  'smoking_status': 'Unknown',
  'stroke': False},
 {'id': 30468,
  'gender': 'Male',
  'age': 58.0,
  'hypertension': True,
  'heart_disease': False,
  'ever_married': True,
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 87.96,
  'bmi': 39.2,
  'smoking_status': 'Never smoked',
  'stroke': False},
 {'id': 46136,
  'gender': 'Male',
  'age': 14.0,
  'hypertension': False,
  'heart_disease': False,
  'ever_married': False,
  'work_type': 'Never worked',
  'Residence_type': 'Rural',
  'avg_glucose_level': 161.28,
  'bmi': 19.1,
  'smoking_status': 'Unknown',
  'stroke': False},
 {'id': 64908,
  'gender': 'Male',
  'age': 79.0,
  'hypertension': False,
  'heart_disease': True,
  'ever_married': True,
  'work_type': 'Private',
  'Residen

A présent on souhaite ajouter des informations sur les types des paramètres et valeurs de retour de la fonction pour faciliter sa compréhension et son utilisation, ce qu’on appelle l’annotation de type (type hinting).

Cette pratique facilite la lecture et la maintenance du code.

Quels changements pour la fonction ?

A la suite de chaque paramètre, on ajoute le type attendu pour le paramètre. À la suite des paramètres on ajoute le type de ce que qui est retourné par la fonction, dans l'exemple ici : 

```def filter_patient(stroke_data_df: pd.DataFrame, gender: str, etc) -> list[dict]```

Ajouter les types dans la définition de la fonction.

Tester la fonction en ne mettant pas de valeur pour *max_age*.

Que se passe-t-il ?

In [17]:
def filter_patient(df_value:pd.DataFrame, max_age, gender:str, stroke:bool):
    
    df_filtered = df_value[(df_value["age"] <= max_age) & 
                        (df_value["gender"] == gender) & 
                        (df_value["stroke"] == stroke)]

    return df_filtered.to_dict('records')

filter_patient(df, gender="Female", stroke=True)

TypeError: filter_patient() missing 1 required positional argument: 'max_age'

Dans la fonction écrite ci-dessus, chaque paramètre est obligatoire. 

On souhaite pouvoir filtrer les patients sur 0, 1 ou 2 des paramètres de la fonction (filtrer seulement sur *max_age*  mais ne pas appliquer de filtres sur _gender_ et _stroke_ par exemple).

On peut rendre optionnel les paramètres d'un fonction en choisissant une valeur par défault. Si on utilise la fonction en n'utilisant pas ces paramètres alors la valeur par défault est utilisé.

Copier coller votre fonction ci-dessous et ajouter en paramètre : `max_age=None`

et ajouter la condition suivante **avant le filtre** sur `max_age` : 

```if max_age is not None : ``` 

Si la fonction _filter_patient_ est appelée sans argument *max_age*, alors le filtre sur *max_age* n'est pas appliqué. 

Il est tout à fait possible de définir une valeur par défault par exemple 30 ans : dans ce cas si la fonction est appelée sans argument *max_age*, alors par défault on filtre les patients ayant moins de 30 ans.

**ATTENTION :** Les paramètres optionnels doivent toujours être à la fin de la liste de paramètres.

In [ ]:
# fonction filter_patient paramètre max_age optionnel
def filter_patient(df_value:pd.DataFrame, gender:str, stroke:bool, max_age:int=None):
    
    if max_age is not None :
        df_filtered = df_value.loc[(df_value["age"] <= max_age) & (df_value["gender"] == gender) & (df_value["stroke"] == stroke)]
        return df_filtered.to_dict('records')
    else:
        df_filtered = df_value.loc[(df_value["gender"] == gender) & (df_value["stroke"] == stroke)]
        return df_filtered.to_dict('records')

In [ ]:
# test fonction sans argument max_age
filter_patient(df, gender="Female", stroke=True)

[{'id': 51676,
  'gender': 'Female',
  'age': 61.0,
  'hypertension': False,
  'heart_disease': False,
  'ever_married': True,
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 202.21,
  'bmi': 61.8,
  'smoking_status': 'Never smoked',
  'stroke': True},
 {'id': 60182,
  'gender': 'Female',
  'age': 49.0,
  'hypertension': False,
  'heart_disease': False,
  'ever_married': True,
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 171.23,
  'bmi': 34.4,
  'smoking_status': 'Smokes',
  'stroke': True},
 {'id': 1665,
  'gender': 'Female',
  'age': 79.0,
  'hypertension': True,
  'heart_disease': False,
  'ever_married': True,
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 174.12,
  'bmi': 24.0,
  'smoking_status': 'Never smoked',
  'stroke': True},
 {'id': 10434,
  'gender': 'Female',
  'age': 69.0,
  'hypertension': False,
  'heart_disease': False,
  'ever_married': False,
  'work_type': 'Pri

Ajouter des valeurs par défault et les conditions pour chaque filtre.

Pour les types, on indique qu'il s'agit de paramètres optionels en utilisant le module python _typing_

```
from typing import Optional
def filter_patient(stroke_data_df: pd.DataFrame, gender: Optional[str] = None,etc)
```

Adapter les types en utilisant ce modèle.

In [ ]:
# fonction avec ajout de paramètres par défault et de type
from typing import Optional

def filter_patient(df_value:pd.DataFrame, gender:Optional[str] = None, stroke:Optional[bool] = None, max_age:Optional[int] = None):
    df_filtered = df_value.copy()
    
    if max_age is not None:
        df_filtered = df_filtered[df_filtered["age"] <= max_age]

    if gender is not None:
        df_filtered = df_filtered[df_filtered["gender"] == gender]

    if stroke is not None:
        df_filtered = df_filtered[df_filtered["stroke"] == stroke]

    return df_filtered.to_dict('records')

Tester la fonction sans argument pour les filtres, elle doit donc renvoyer le dataframe non filtré.

In [ ]:
# test fonction sans argument pour les filtres
filter_patient(df)

[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  'hypertension': False,
  'heart_disease': True,
  'ever_married': True,
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 228.69,
  'bmi': 36.6,
  'smoking_status': 'Formerly smoked',
  'stroke': True},
 {'id': 51676,
  'gender': 'Female',
  'age': 61.0,
  'hypertension': False,
  'heart_disease': False,
  'ever_married': True,
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 202.21,
  'bmi': 61.8,
  'smoking_status': 'Never smoked',
  'stroke': True},
 {'id': 31112,
  'gender': 'Male',
  'age': 80.0,
  'hypertension': False,
  'heart_disease': True,
  'ever_married': True,
  'work_type': 'Private',
  'Residence_type': 'Rural',
  'avg_glucose_level': 105.92,
  'bmi': 32.5,
  'smoking_status': 'Never smoked',
  'stroke': True},
 {'id': 60182,
  'gender': 'Female',
  'age': 49.0,
  'hypertension': False,
  'heart_disease': False,
  'ever_married': True,
  'work_type': 'Privat

Cette fonction va être utilisée dans la définition de l'API pour créer une route qui permette d'accéder à des données filtrées sur les patients.

Dans le fichier de définition de l'API, toutes les fonctions vont travailler sur les données du fichier. 

Pour alléger les fonctions on va donc utiliser une **variable globale** pour les données et supprimer le paramètre `df` de la fonction.

On lit les données en début de fichier puis on travaille au sein des fonctions sur une copie du dataframe de données.


**En résumé les modifications à faire sont :**


- Supprimer le paramètre df de la fonction,
- Ajouter en début de fonction :  
```df = stroke_data_df.copy()```

1. Dans le fichier filters.py, il suffit d'ajouter : 
- lecture du fichier de données prétraitée dans la variable *df* en début de fichier (utiliser pandas),
- @app.get("/patients/") pour définir le route,
puis la fonction.

2. Dans le fichier api.py: appeler la fonction dans la route correspondante.

Tester la route avec 

```poetry run fastapi dev stroke_api/main.py```

http://127.0.0.1:8000/docs : utiliser la fonctionnalité Try it out pour tester la route.

---
## Autres routes

De la même manière, créer les fonctions appropriées pour la création de :
- la route `/patients/{id}` : Récupère les détails d’un patient donné (via son identifiant unique) 

- la route `/stats/` : Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes).

- Lister les tâches à faire sous forme d'issue github : travailler sur une branche différentes pour l'ajout de chacune des routes.

STATS

In [ ]:
def stats(type:Optional[str] = None):
    df1 = df.copy()

    if type is None:
        fig  = px.pie()

In [ ]:
def filter_id(id_patient:int):
    df1 = df.copy()

    if id_patient in df1["id"].tolist():
        return df1.loc[df1["id"] == id_patient].to_dict('records')
    else:
        return print("Ce patient id n'existe pas")
    

In [ ]:
filter_id(2)

Ce patient id n'existe pas


In [23]:
nb_patient = df["id"].count()
age_moyen =  df["age"].mean()
avc_moyen = df["stroke"].mean()

print(f"{round(nb_patient)} patients, ", f"{round(age_moyen)} ans en moyenne, ", f"{avc_moyen}% taux d'avc")

5110 patients,  43 ans en moyenne,  0.0487279843444227% taux d'avc
